In [1]:
import numpy as np
import pandas as pd
import math
from itertools import combinations

In [2]:
salesdf=pd.read_excel('C:\\Users\\amirarzy\\Dropbox\\job\\sportkala\\Python\\recommender.xlsx',sheet_name='Sales')

In [3]:
recommenddf=pd.read_excel('C:\\Users\\amirarzy\\Dropbox\\job\\sportkala\\Python\\recommender.xlsx',sheet_name='recommend items')

In [4]:
productdf=pd.read_excel('C:\\Users\\amirarzy\\Dropbox\\job\\sportkala\\Python\\recommender.xlsx',sheet_name='products')

In [5]:
invoice_matrix=salesdf.pivot_table(index='invoice',columns='group',values='qty')

In [6]:
corr_matrix=invoice_matrix.corr().fillna(0)

In [7]:
freq_list=[(col1,col2,invoice_matrix[(invoice_matrix[col1].notna()) & (invoice_matrix[col2].notna())].shape[0]) for (col1,col2) in combinations (invoice_matrix.columns,2)]

In [8]:
freq_index=list()

In [9]:
freq_index.append(freq_list[0][0])
for i in range(0,len(corr_matrix)-1):
    freq_index.append(freq_list[i][1])

In [10]:
freqdf = pd.DataFrame(0,columns=freq_index,index=freq_index)

In [11]:
k=1
n=1
for j in range(0,len(corr_matrix)):
    for i in range(n,len(corr_matrix)):
        freqdf.iloc[j][i]=freq_list[k-1][2]
        k=k+1
    n=n+1
j=0
m=1
for j in range(0,len(corr_matrix)):
    for i in range(m,len(corr_matrix)):
        freqdf.iloc[i][j]=freqdf.iloc[j][i]
    m=m+1  

In [36]:
performidlist=['20100000184','20100000093','20100000149']

In [37]:
performlist=list()
for r in range(len(performidlist)):
    if len(productdf.loc[productdf['id']==int(performidlist[r])])>0:
        performlist.append(productdf.loc[productdf['id']==int(performidlist[r])]['group'].to_string(index=False))

In [38]:
performlist=list(set(performlist))
performlist

['کیف پزشکی', 'دست', 'مچ پا']

In [39]:
scorepd=list()
for l in range(len(performlist)):
    scorecorr=corr_matrix.loc[corr_matrix[performlist[l]]>0][performlist[l]]
    scorefreq=freqdf.loc[freqdf[performlist[l]]>0][performlist[l]]
    scorepd.append(pd.concat([scorecorr, scorefreq], axis=1).fillna(0))  

In [40]:
for x in range(len(performlist)):
    scorepd[x].columns =['corr','freq']
result=scorepd[0]
if len(performlist)-1>0:
    for y in range(len(performlist)-1):
        result=result.append(scorepd[y+1])       

C:\Users\amirarzy\AppData\Local\Temp\ipykernel_2736\1256216114.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result=result.append(scorepd[y+1])
C:\Users\amirarzy\AppData\Local\Temp\ipykernel_2736\1256216114.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result=result.append(scorepd[y+1])


In [41]:
scorepd=result.groupby(result.index)['corr','freq'].sum()

C:\Users\amirarzy\AppData\Local\Temp\ipykernel_2736\1516716766.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  scorepd=result.groupby(result.index)['corr','freq'].sum()


In [42]:
scorepd['score']=scorepd['corr']*scorepd['freq']

In [43]:
scorepd.sort_values(by=['score','freq'],inplace=True,ascending=False)

In [44]:
scorepd.head(30)

,corr,freq,score
کنزیوتیپ,0.997801,5.0,4.989007
جیم بال,0.000000,4.0,0.000000
مینی بند,0.000000,4.0,0.000000
توپ ماساژ,0.000000,3.0,0.000000
سوت,0.000000,3.0,0.000000
اسپری,0.000000,2.0,0.000000
بوتی بند,0.000000,2.0,0.000000
تلمبه,0.000000,2.0,0.000000
حلقه چابکی,0.000000,2.0,0.000000
دیجی فلکس,0.000000,2.0,0.000000


In [45]:
z=0
finalrecommendlist=list()
while len(finalrecommendlist)<3 and z<len(scorepd):
    recommend_list=recommenddf.loc[recommenddf['group']==scorepd.index[z]]
    if len(recommend_list)>0:
        final_recommend=recommend_list.sample(weights='weight')['dsc'].to_string(index=False)
        finalrecommendlist.append(final_recommend)
    z=z+1

In [46]:
finalrecommendlist

['صادق', 'ارضی', 'پدرجان']